<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/fine_tuning_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning Embedding Models

In the following Notebook we will be exploring one of the most powerful techniques to take your single-domain RAG pipelines to the next level...

## Task 1: Dependencies and Boilerplate

We'll set up our `nest_asyncio` so we can leverage async loops in our Notebook.

We'll also install the required libraries we'll be using today, and set up our OpenAI API key!

### Nest Asyncio

In [1]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies

In [2]:
!pip install -qU langchain_mistralai langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 622.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5

In [3]:
!pip install -qU faiss-cpu unstructured==0.15.7 python-pptx==1.0.2 nltk==3.9.1 openparse[ml] html2text langchain langchain_groq langchain_huggingface faiss-cpu langchain-pinecone pinecone-notebooks langchain_mistralai
!mkdir -p data
!wget https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico -O "data/ppc_2018.pdf"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 33.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298

In [4]:
from google.colab import userdata

class Settings:
  HF_TOKEN          = userdata.get('HF_TOKEN')
  PINECONE_API_KEY  = userdata.get('PINECONE_API_KEY')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')
  MISTRAL_AI_KEY    = userdata.get('MISTRAL_AI_KEY')
  GROQ_API_KEY2     = userdata.get('GROQ_API_KEY2')
  MISTRAL_AI_KEY2   = userdata.get('MISTRAL_AI_KEY2')


settings = Settings()

## Task 2: Loading Data

The data can be found in [this GitHub repo](https://github.com/AI-Maker-Space/DataRepository/tree/main/high-performance-rag).

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!cp /content/drive/My\ Drive/tcc/PPC_2024.pdf data/ppc_2024.pdf

In [6]:
from langchain_core.load import dumpd, dumps, load, loads
import json
import time
import hashlib
import os


def save_document_locally(documents, version):

  output_file_name = f'ppc{version}_documents.json'
  json_string = dumps(documents, pretty=True, ensure_ascii=False)
  with open(output_file_name, "w", encoding="utf-8") as file:
      file.write(json_string)

def load_document_locally(version):
  output_file_name = f'ppc{version}_documents.json'
  with open(output_file_name, "r") as fp:
      doc = json.load(fp)
  return load(doc)


def save_dict_to_json_on_drive(documents, drive_folder_path, drive_file_name):

    os.makedirs(drive_folder_path, exist_ok=True)
    timestamp = str(int(time.time()))
    hash_suffix = hashlib.md5(timestamp.encode()).hexdigest()[:6]  # Gera um hash curto com 6 caracteres

    drive_file_name = f'{hash_suffix}_{drive_file_name}'
    drive_path = f'{drive_folder_path}/{os.path.basename(drive_file_name)}'

    json_string = dumpd(documents)
    print(json_string)
    with open(drive_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_string, json_file, ensure_ascii=False)
    print(f'File saved to: {drive_path}')
    return drive_path

In [8]:
import openparse
import html2text
from langchain_core.documents import Document
from uuid import uuid4


def parse_document_to_documents(file_path, source_name):
    # Parse the document using OpenParse
    parser = openparse.DocumentParser(
                                      table_args = {"parsing_algorithm": "pymupdf", "table_output_format": "markdown"}
                                      )
    document = parser.parse(file_path)

    documents = []
    ids = []

    for node in document.nodes:

        html_content = node.text

        if html_content:
            id = str(uuid4())
            ids.append(id)
            page = node.bbox[0].page + 1
            document_chunk = Document(
                metadata={'source': source_name,
                          'id': str(uuid4()),
                          'page': page},
                page_content=html_content
            )
            documents.append(document_chunk)

    return documents, ids, document


In [19]:
documents_2018, ids_2018, nodes_2018 = parse_document_to_documents('data/ppc_2018.pdf','ppc2018')
documents_2024, ids_2024, nodes_204 = parse_document_to_documents('data/ppc_2024.pdf','ppc2024')

In [20]:
training_documents = documents_2018 + documents_2024

In [21]:
len(training_documents)

923

In [22]:
training_split_documents = training_documents[:300]

In [23]:
val_split_documents = training_documents[300:350]

In [24]:
test_split_documents = training_documents[350:400]

## Task 3: Constructing a Fine-tuning Dataset

Using the nodes we created above, we can finally start constructing a fine-tuning dataset utilizing OpenAI's `gpt-4o-mini` (released [today](https://openai.com/index/gpt-4o-mini-advancing-cost-efficient-intelligence/)).

The basic idea here is straightforward enough:

1. We look at a document
2. We generate questions that could be answered by that node

This gives us a number of question/context pairs that we can use to fine-tune our Embeddings model.

In [25]:
from langchain_mistralai import ChatMistralAI

qa_chat_model = ChatMistralAI(model='open-mistral-7b',
                    temperature=1,
                    max_retries=4,
                    api_key=settings.MISTRAL_AI_KEY2)

We'll create a simple Question Generation prompt to query `gpt-4o-mini` to generate Questions for each retrieved context.

In [26]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Dado o seguinte contexto, você deve gerar perguntas com base apenas no contexto fornecido.

Você deve gerar {n_questions} perguntas que devem ser apresentadas no seguinte formato:

PERGUNTA #1
PERGUNTA #2 ...
Contexto: {context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

We'll create a simple chain to query the LLM!

In [27]:
question_generation_chain = qa_prompt_template | qa_chat_model

There's a lot going on in this function - let's take a deeper look:

1. First, we provide a list of documents and a number of questions
2. We, for each document in our list, generate `n_questions` of questions.
3. We then associate those questions and contexts via a `UUID`.

> NOTE: The reason we're doing this `UUID` association is for ease of use later in the notebook.

In [34]:
import tqdm
import uuid
import time

def create_questions(documents, n_questions):
    questions = {}
    relevant_docs = {}
    for document in tqdm.tqdm(documents):
        document_content = {"context": document.page_content, "questions": []}
        attempt = 0

        while attempt < 3:  # Tente até 3 vezes
            try:
                time.sleep(1)
                questions_generated = question_generation_chain.invoke({"context": document.page_content, "n_questions": n_questions})
                print(questions_generated)

                for question in questions_generated.content.split("\n"):
                    question_id = str(uuid.uuid4())
                    questions[question_id] = "".join(question.split(".")[1:]).strip()
                    relevant_docs[question_id] = [document.metadata["id"]]
                break  # Se a geração de perguntas for bem-sucedida, saia do loop

            except Exception as e:
                print(f"Erro ao gerar perguntas: {e}. Tentando novamente em 15 segundos...")
                time.sleep(15)  # Aguarde 15 segundos antes de tentar novamente
                attempt += 1  # Incrementa o número de tentativas

    return questions, relevant_docs


We'll use the function to generate training, validation, and test data.

In [35]:
training_questions, training_relevant_contexts = create_questions(training_split_documents, 10)

  0%|          | 1/300 [00:03<16:50,  3.38s/it]

content='1. O ano em que a Universidade Federal da Fronteira Sul (UFFS) foi criada?\n2. Qual é a localização da sede da UFFS?\n3. Quantos *campi* tem a UFFS, sendo eles nos quais estados?\n4. Como se chama a lei que acredita a criação da UFFS?\n5. Em quais áreas de conhecimento a UFFS oferece graduação?\n6. Qual é a função da Pró-reitoria de Graduação na UFFS?\n7. Qual o papel da Diretoria de Pedagógica na UFFS?\n8. Como a UFFS se dividiu em "abrangência interestadual"?\n9. Por quais motivos a UFFS possui *campi* nos estados do Rio Grande do Sul e Paraná?\n10. Quantos estudantes atualmente estão matriculados na UFFS? (Não há informação suficiente para responder esta pergunta)' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 305, 'total_tokens': 555, 'completion_tokens': 250}, 'model': 'open-mistral-7b', 'finish_reason': 'stop'} id='run-bfecedd6-af5c-4362-8b9d-b1a69fa3a583-0' usage_metadata={'input_tokens': 305, 'output_tokens': 250, 'total_tokens': 555}


  1%|          | 2/300 [00:06<16:43,  3.37s/it]

content='1. Qual é a avenida onde se localiza a Reitoria?\n2. Qual é o código postal do bairro Centro onde está a Reitoria?\n3. Quem atua como reitor na instituição?\n4. Qual é o nome do vice-reitor da instituição?\n5. Quem assume a função de Pró-Reitor de Graduação na instituição?\n6. Quem é o Pró-Reitor de Pesquisa e Pós-Graduação da instituição?\n7. Qual é o nome do Pró-Reitor de Extensão e Cultura?\n8. Quem atua como Pró-Reitor de Administração e Infraestrutura na instituição?\n9. Quem é o Pró-Reitor de Planejamento da instituição?\n10. Qual é o nome do Pró-Reitor de Assuntos Estudantis na instituição?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 354, 'total_tokens': 588, 'completion_tokens': 234}, 'model': 'open-mistral-7b', 'finish_reason': 'stop'} id='run-e1a95e60-7f03-4706-b408-eb1eae2a0fcb-0' usage_metadata={'input_tokens': 354, 'output_tokens': 234, 'total_tokens': 588}


  1%|          | 3/300 [00:10<17:03,  3.44s/it]

...

content='1. O que são as atividades complementares no curso de Graduação em Ciência da Computação - Bacharelado?\n2. Qual tipo de regulamento regulamenta as atividades complementares do curso de Ciência da Computação?\n3. Podem ser realizadas as atividades complementares antes da conclusão do curso?\n4. Existem limitação quanto ao número de cursos complementares que podem ser cursados?\n5. Qual é o objetivo das atividades complementares no curso de Ciência da Computação?\n6. As atividades complementares são obrigatórias?\n7. Podem ser realizadas as atividades complementares durante o período de férias escolares?\n8. Existem requisitos específicos para a realização das atividades complementares?\n9. Quais são as disciplinas ou áreas cursadas nas atividades complementares?\n10. Como são creditados as atividades complementares no transcrição acadêmica?' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 170, 'total_tokens': 411, 'completion_tokens': 241}, 'model': 'o

100%|█████████▉| 299/300 [23:07<00:08,  8.44s/it]

content='1. Qual é o objetivo do presente Regulamento?\n2. Qual curso de graduação é aplicável ao presente Regulamento?\n3. Em que cidade a Universidade que regula o presente Regulamento está localizada?\n4. O que deve ser normatizado pelas Disposições Gerais do presente Regulamento?\n5. Quais são as normas gerais de controle, aproveitamento, validação e registro das horas complementares estabelecidas pelo Regulamento?\n6. Qual é a carga horária admitida para cada atividade válida para as Atividades Curriculares Complementares?\n7. Quais são as Atividades Curriculares Complementares obrigatórias do curso de Ciência da Computação?\n8. Quais são os documentos comprobatórios necessários para o cumprimento do presente Regulamento?\n9. Quais são os períodos de realização das Atividades Curriculares Complementares?\n10. Quão exatamente é a carga horária correspondente aos 20 créditos das Atividades Curriculares Complementares?' additional_kwargs={} response_metadata={'token_usage': {'prompt_

100%|██████████| 300/300 [23:10<00:00,  4.64s/it]

content='1. O que definem as atividades válidas no curso de Ciência da Computação?\n2. Quando a validação das atividades do aluno começa no curso de Ciência da Computação?\n3. Como são avaliadas as atividades desenvolvidas pelos alunos no curso de Ciência da Computação?\n4. Qual o critério primário a ser considerado na validação das atividades do aluno no curso de Ciência da Computação?\n5. Como a formação do futuro bacharel em Ciência da Computação é afetada pela validação das atividades do aluno?\n6. Qual é o papel da Comissão de Atividades Curriculares Complementares no curso de Ciência da Computação?\n7. Quais outros critérios são analisados e definidos pela Comissão de Atividades Curriculares Complementares no curso de Ciência da Computação?\n8. Como as atividades devem se correlacionar com os objetivos do curso de Ciência da Computação?\n9. Como as atividades desenvolvidas pelos alunos contribuem para a formação de profissionais de Ciência da Computação?\n10. Qual é a importância

In [36]:
val_questions, val_relevant_contexts = create_questions(val_split_documents, 10)

  2%|▏         | 1/50 [00:04<03:33,  4.36s/it]

content='PERGUNTA #1\nQual é o propósito geral das Atividades Curriculares Complementares (ACC) de Graduação do curso de Ciência da Computação?\nPERGUNTA #2\nQual é o objetivo específico do item I das ACC no curso de Ciência da Computação?\nPERGUNTA #3\nComo a flexibilização do currículo obrigatório das ACC contribui para o curso de Ciência da Computação?\nPERGUNTA #4\nExplique por que a flexibilização do currículo obrigatório das ACC é importante para o curso de Ciência da Computação.\nPERGUNTA #5\nEspecifique três vantagens que a models de ACC podem oferecer ao curso de Ciência da Computação.\nPERGUNTA #6\nQual é o papel das ACC no Ensino Superior de Ciência da Computação?\nPERGUNTA #7\nComo as Atividades Curriculares Complementares (ACC) de Graduação impactam a formação de estudantes no curso de Ciência da Computação?\nPERGUNTA #8\nQual é a importância das ACC em relação à formação profissional dos estudantes do curso de Ciência da Computação?\nPERGUNTA #9\nQual o papel da universid

  4%|▍         | 2/50 [00:08<03:29,  4.37s/it]

content='1. Qual é a função primária da Proreitoria de Graduação na Diretoria de Pedagógica do Ministério da Educação da UFFS?\n2. Como essa proreitoria aproxima o estudante da realidade social e profissional?\n3. Qual é o propósito da participação do universitário em atividades visando a formação profissional e para a cidadania?\n4. Como a Diretoria de Pedagógica promove o aprofundamento temático e interdisciplinar na universidade?\n5. Como essa diretoria propicia a integração entre a universidade e a sociedade?\n6. Como a participação dos estudantes na formação profissional e para a cidadania ocorre na UFFS?\n7. Qual o papel que o Ministério da Educação desempenha na Proreitoria de Graduação na Diretoria de Pedagógica da UFFS?\n8. Como a participação de atividades em prol da formação profissional e cidadania contribui para melhorar a educação na UFFS?\n9. Qual o benefício de promover a integração entre a universidade e a sociedade ao estudante?\n10. Como a Proreitoria de Graduação na

  6%|▌         | 3/50 [00:12<03:18,  4.23s/it]

...

content='1. Qual é a entidade responsável por elaborar a introdução, procedimentos metodológicos, resultados e conclusões do trabalho?\n2. O que é expectado em relação aos documentos anexos para que sejam reconhecidos e aceitos como relevantes e válidos?\n3. Por quais partes interessadas é necessário que os documentos anexos sejam contextualizados?\n4. Qual é a pontuação mínima necessária para que uma compilação seja aceita para defesa?\n5. As pontuações dos documentos anexos podem ser diferentes, mas seguirem critérios definidos em qual tabela?\n6. Como se dão os critérios para definir a pontuação dos documentos anexos diferentes de mesma natureza?\n7. Qual é o papel do Colegiado do Curso ou da Comissão de Atividades Curriculares Complementares na avaliação dos documentos anexos?\n8. O texto que explica como os desdobramentos dos documentos anexos se conectam na contribuição do trabalho é necessário em todo documento anexo? Por que?\n9. Como as contribuições dos documentos anexos são 

 98%|█████████▊| 49/50 [03:25<00:04,  4.14s/it]

content='PERGUNTA #1\nQual a redução do número de membros de um time finalista que poderão inscrever relatórios de análise do mesmo problema na etapa Nacional de Maratona de Programação?\n\nPERGUNTA #2\nQuantos pontos são atribuídos ao relatório de participação de um membro titular da equipe na etapa Regional de Maratona de Programação?\n\nPERGUNTA #3\nQual a condição necessária para que um aluno seja considerado como titular de uma equipe na etapa Regional de Maratona de Programação?\n\nPERGUNTA #4\nQual é o mínimo de duração mínima para que um projeto de Iniciação Científica possa ter seu relatório com pontuação de 50 pontos na tabela 3?\n\nPERGUNTA #5\nQual o tipo de evento definido na tabela 3 que possui resumos ou equivalentes aceitos e publicados com apenas 5 pontos?\n\nPERGUNTA #6\nQual a duração mínima de um projeto conduzido por grupo(s) institucionalizados para receber 15 pontos na tabela 3?\n\nPERGUNTA #7\nO que se entende por projeto de Iniciação Científica?\n\nPERGUNTA #8\

100%|██████████| 50/50 [03:29<00:00,  4.19s/it]

content='1. Como se define o projeto conduzido pelos grupos institucionalizados do curso?\n2. Qual a duração mínima para o projeto que faz parte deste relatório?\n3. O produto entregável deste projeto deve ser publicado para quem?\n4. Quem deve dar o reconhecimento ao produto entregável do projeto?\n5. Qual a função do professor mentor na atividade deste projeto?\n6. No relatório de participação, quais informações devem ser apresentadas sobre cada etapa do processo de desenvolvimento de projeto de Emprendedorismo ou Inovação?\n7. Em qual fase do processo de desenvolvimento de projeto de Emprendedorismo ou Inovação, deverão ser apresentados documentos entregáveis?\n8. Qual é a finalidade de validar a ideia no processo de desenvolvimento de projeto de Emprendedorismo ou Inovação?\n9. Qual o objetivo de definir e executar o MVP no processo de desenvolvimento de projeto de Emprendedorismo ou Inovação?\n10. Em qual disciplina os documentos apresentados no processo de desenvolvimento de proj

In [37]:
test_questions, test_relevant_contexts = create_questions(test_split_documents, 10)

  2%|▏         | 1/50 [00:04<03:24,  4.17s/it]

content='PERGUNTA #1\nQual a condição necessária para integrar o trabalho digital a UFFS?\n\nPERGUNTA #2\nQual a linguagem obrigatória para o documento produzido no TCC II?\n\nPERGUNTA #3\nQuais são as atribuições da coordenação de curso no TCC II?\n\nPERGUNTA #4\nQual é a responsabilidade do professor responsável pelo componente curricular TCC I ou TCC II?\n\nPERGUNTA #5\nQual é o idioma obrigatório para o documento produzido no TCC II de acordo com o artigo 17º?\n\nPERGUNTA #6\nQuais as normas metodológicas que devem ser seguidas pelo estudante no TCC II?\n\nPERGUNTA #7\nQual o papel da universidade na divulgação do conhecimento adquirido através dos trabalhos digitais dos estudantes na UFFS?\n\nPERGUNTA #8\nQuais seriam as atribuições do professor orientador no curso de TCC II?\n\nPERGUNTA #9\nDe que forma serão supervisionadas as questões ligadas aos componentes curriculares de TCC em geral?\n\nPERGUNTA #10\nQual a última etapa do processo de avaliação dos trabalhos no TCC II?' add

  4%|▍         | 2/50 [00:08<03:23,  4.24s/it]

content='PERGUNTA #1\nQuais são os procedimentos e normas que serão esclarecidos durante os encontros com os estudantes para a realização do TCC?\n\nPERGUNTA #2\nO que consiste no cronograma semestral do componente curricular e qual o período de defesas dos TCCs?\n\nPERGUNTA #3\nQuais os trâmites burocráticos relacionados ao TCC serão administrados?\n\nPERGUNTA #4\nQual papel o docente desempenha no repassamento das normas de metodologia aos estudantes na confecção dos documentos para o TCC?\n\nPERGUNTA #5\nQuando os alunos devem entregar os materiais para o TCC conforme os estabelecidos nos encontros?\n\nPERGUNTA #6\nQuais são as outras atividades que são gerenciadas durante o processo de TCC?\n\nPERGUNTA #7\nQual a importância do projeto pedagógico no Curso de Graduação em Ciência da Computação – Bacharelado 219?\n\nPERGUNTA #8\nQual é a estrutura geral do projeto pedagógico do Curso de Graduação em Ciência da Computação – Bacharelado 219?\n\nPERGUNTA #9\nQuais benefícios os alunos p

  6%|▌         | 3/50 [00:12<03:14,  4.15s/it]

...

content='1. Qual componente curricular está condicionado à entrega de uma versão final da monografia ou artigo?\n2. Em que forma a aprovação no componente curricular TCC II está condicionada à entrega do trabalho?\n3. Qual biblioteca está responsável por preconizar o padrão e normas para a entrega da monografia ou artigo na UFFS?\n4. Qual é a referência (Memório 014/2014-PROGRAD) que fornece as diretrizes para o padrão e normas para a entrega da monografia ou artigo na UFFS?\n5. Qual tipo de trabalho é considerado quando não se entrega uma versão final da monografia ou artigo, mas sim o artigo científico publicado?\n6. Como deve ser entregue a versão final da monografia ou artigo, de acordo com as normas preconizadas pela biblioteca da UFFS?\n7. Quando a versão final da monografia ou artigo é publicada em um artigo científico, qual critério deve ser seguido para ser considerada válida para a aprovação no componente curricular TCC II na UFFS?\n8. Qual a importância da entrega da versão 

 98%|█████████▊| 49/50 [03:23<00:03,  3.99s/it]

content='1. Qual é o artigo específico que se refere à determinação de casos omissos neste texto?\n2. Qual o corpo consultivo responsável por decidir sobre casos omissos no texto referenciado no artigo 20?\n3. Como são definidos os casos omissos no artigo 20 do texto apresentado?\n4. Qual é a finalidade do artigo 20 em relação aos casos omissos no texto com o qual se trata?\n5. São dispensados alguns casos específicos no artigo 20 do texto apresentado?\n6. Como os casos omissos são determinados pelo Colegiado de Curso, conforme o artigo 20?\n7. Qual o caráter legal dos casos omissos no artigo 20 do texto?\n8. É uma responsabilidade do Colegiado de Curso decidir sobre casos omissos nos textos gerais ou transitórios?\n9. Há alguma limitação ao poder do Colegiado de Curso em relação aos casos omissos no artigo 20 do texto apresentado?\n10. Como os casos omissos no projeto ou ato jurídico gerido pelos artigos são fiscalizados e interpretados pelos órgãos competentes, como o Colegiado de Cu

100%|██████████| 50/50 [03:27<00:00,  4.14s/it]

content='1. Qual tipo de regulamento se refere ao Artigo 21 na questão?\n2. Qual curso está relacionado àquele Regulamento de Trabalho de Conclusão de Curso?\n3. O que acontece com o Regulamento de Trabalho de Conclusão de Curso após sua aprovação na CGRAD?\n4. Qual outra entidade está associada à aprovação do regimento mencionado?\n5. Em que campo de estudo o Regulamento de Trabalho de Conclusão de Curso é aplicado?\n6. Quais saídas podem surgir após a aprovação do Regulamento de Trabalho de Conclusão de Curso pelo PPC?\n7. Existe alguma responsabilidade que um trabalhador tem em relação ao Regulamento de Trabalho de Conclusão de Curso?\n8. Como pode ser o Regulamento de Trabalho de Conclusão de Curso influenciado após seu aprovação?\n9. O Regulamento de Trabalho de Conclusão de Curso é um documento em vigor ou em projeto no momento do contexto?\n10. Quais podem ser as conseqüências negativas de falta de cumprimento do Regulamento de Trabalho de Conclusão de Curso?' additional_kwargs=

We'll save each dataset for use later.

> NOTE: These datasets will be provided in the repository in case you run into any issues with the data generation steps or you wish to save API calls.

In [38]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [39]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [40]:
train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

In [47]:
paths = [save_dict_to_json_on_drive(train_dataset, '/content/drive/MyDrive/tcc/preprocessing/ft', 'train_dataset.json'),
save_dict_to_json_on_drive(val_dataset, '/content/drive/MyDrive/tcc/preprocessing/ft', 'val_dataset.json'),
save_dict_to_json_on_drive(test_dataset, '/content/drive/MyDrive/tcc/preprocessing/ft', 'test_dataset.json')]

{'questions': {'b3ec9576-1e8f-4e56-a83a-aa04d6794f86': 'O ano em que a Universidade Federal da Fronteira Sul (UFFS) foi criada?', 'fcd20467-d43e-43b0-9d3c-ae0c5f42609f': 'Qual é a localização da sede da UFFS?', 'a8195cdf-6ff1-4196-ad09-9c886586fd7b': 'Quantos *campi* tem a UFFS, sendo eles nos quais estados?', '537e3cbf-0e96-4a5d-a578-368a1c917bb5': 'Como se chama a lei que acredita a criação da UFFS?', '07167844-f052-4fa2-ae1f-b6a74589d84c': 'Em quais áreas de conhecimento a UFFS oferece graduação?', '251f7495-c5b7-4f14-9751-60110eda3f57': 'Qual é a função da Pró-reitoria de Graduação na UFFS?', 'ac4ff84a-696c-4841-ba8d-87e5b9754df2': 'Qual o papel da Diretoria de Pedagógica na UFFS?', '370f8781-52f7-4486-bf3b-07affa0c606f': 'Como a UFFS se dividiu em "abrangência interestadual"?', '616c8d16-3e16-4377-8b39-378dde74af97': 'Por quais motivos a UFFS possui *campi* nos estados do Rio Grande do Sul e Paraná?', 'a5142ab4-6038-4cc2-a571-66fb9a718f05': 'Quantos estudantes atualmente estão mat

In [48]:
paths

['/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_train_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_val_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_test_dataset.json']

In [50]:

['/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_train_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_val_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_test_dataset.json']

['/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_train_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_val_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_test_dataset.json']

## Task 4: Fine-tuning `snowflake-arctic-embed-m`

Now that we have a dataset, let's grab a `sentence-transformers` Embeddings model!

We'll be using Snowflake's [`snowflake-arctic-embed-m`](https://huggingface.co/Snowflake/snowflake-arctic-embed-m) as a base embeddings model.

It is a well performing embeddings model by itself, but there's a lot of very specific domain terms and vocabulary in our courpus - so lets fine-tune it and see what that can do for us!

In [7]:
# prompt: load the files within paths

import json

def load_json_from_drive(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return data
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {file_path}")
        return None


paths = ['/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_train_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_val_dataset.json',
 '/content/drive/MyDrive/tcc/preprocessing/ft/bf23bc_test_dataset.json']

loaded_data = []
for path in paths:
  loaded_data.append(load_json_from_drive(path))

train_dataset = loaded_data[0]
val_dataset = loaded_data[1]
test_dataset = loaded_data[2]

In [8]:
!pip install -qU sentence_transformers datasets pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00


In [9]:
from sentence_transformers import SentenceTransformer

model_id = "sentence-transformers/distiluse-base-multilingual-cased-v1"
model = SentenceTransformer(model_id)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

We'll grab some necessary imports from `sentence_transformers` and `torch`.

In [10]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

We're using a toy batch size here to reflect the limited number of examples we have.

Ideally we'd want to use a much larger batch size. (~64+)

In [11]:
BATCH_SIZE = 20

Let's move our dataset into the expected format for training.

In [12]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

Now we can create a `torch` `DataLoader`!

In [13]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

Next up, we'll prepare our loss function!

The loss we're using today is called `MultipleNegativesRankingLoss` - you can find more information [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MultipleNegativesRankingLoss.py).

In [18]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

Now we can set-up our evaluator.

> NOTE: Due to the formatting of our dataset - this is all we have to do!

In [15]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

We'll train this model for 5 epochs, though you could increase this number if we had a significant amount more data.

In [16]:
EPOCHS = 10

It's training time!

> NOTE: We're manually defining a warm-up period here - this is just to provide a smooth ramp into our training!

In [19]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned-v2-distiluse-base-multilingual-cased-v1',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50,
)

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100,Dot Accuracy@1,Dot Accuracy@3,Dot Accuracy@5,Dot Accuracy@10,Dot Precision@1,Dot Precision@3,Dot Precision@5,Dot Precision@10,Dot Recall@1,Dot Recall@3,Dot Recall@5,Dot Recall@10,Dot Ndcg@10,Dot Mrr@10,Dot Map@100
50,No log,No log,0.219072,0.309278,0.337629,0.454897,0.219072,0.103093,0.067526,0.045490,0.219072,0.309278,0.337629,0.454897,0.321735,0.281536,0.306766,0.220361,0.315722,0.336340,0.447165,0.220361,0.105241,0.067268,0.044716,0.220361,0.315722,0.336340,0.447165,0.319654,0.281056,0.306626
100,No log,No log,0.213918,0.349227,0.399485,0.470361,0.213918,0.116409,0.079897,0.047036,0.213918,0.349227,0.399485,0.470361,0.336198,0.293623,0.319230,0.203608,0.338918,0.394330,0.463918,0.203608,0.112973,0.078866,0.046392,0.203608,0.338918,0.394330,0.463918,0.326589,0.283073,0.308199
150,No log,No log,0.228093,0.377577,0.429124,0.476804,0.228093,0.125859,0.085825,0.047680,0.228093,0.377577,0.429124,0.476804,0.351164,0.310819,0.335313,0.216495,0.368557,0.423969,0.465206,0.216495,0.122852,0.084794,0.046521,0.216495,0.368557,0.423969,0.465206,0.340722,0.300549,0.325590
187,No log,No log,0.221649,0.331186,0.404639,0.475515,0.221649,0.110395,0.080928,0.047552,0.221649,0.331186,0.404639,0.475515,0.340259,0.297820,0.321452,0.210052,0.328608,0.395619,0.467784,0.210052,0.109536,0.079124,0.046778,0.210052,0.328608,0.395619,0.467784,0.334080,0.291701,0.315535
200,No log,No log,0.220361,0.371134,0.422680,0.525773,0.220361,0.123711,0.084536,0.052577,0.220361,0.371134,0.422680,0.525773,0.363380,0.312647,0.332610,0.215206,0.367268,0.420103,0.521907,0.215206,0.122423,0.084021,0.052191,0.215206,0.367268,0.420103,0.521907,0.357651,0.306674,0.326667
250,No log,No log,0.220361,0.335052,0.417526,0.532216,0.220361,0.111684,0.083505,0.053222,0.220361,0.335052,0.417526,0.532216,0.360693,0.307946,0.327919,0.220361,0.355670,0.409794,0.533505,0.220361,0.118557,0.081959,0.053351,0.220361,0.355670,0.409794,0.533505,0.360410,0.307430,0.327029
300,No log,No log,0.219072,0.356959,0.420103,0.500000,0.219072,0.118986,0.084021,0.050000,0.219072,0.356959,0.420103,0.500000,0.349076,0.302047,0.322895,0.207474,0.346649,0.407216,0.496134,0.207474,0.115550,0.081443,0.049613,0.207474,0.346649,0.407216,0.496134,0.341609,0.293272,0.314556
350,No log,No log,0.259021,0.367268,0.427835,0.503866,0.259021,0.122423,0.085567,0.050387,0.259021,0.367268,0.427835,0.503866,0.370558,0.328980,0.352213,0.250000,0.351804,0.417526,0.498711,0.250000,0.117268,0.083505,0.049871,0.250000,0.351804,0.417526,0.498711,0.360507,0.317681,0.341169
374,No log,No log,0.221649,0.364691,0.417526,0.505155,0.221649,0.121564,0.083505,0.050515,0.221649,0.364691,0.417526,0.505155,0.353308,0.306161,0.328905,0.220361,0.354381,0.409794,0.500000,0.220361,0.118127,0.081959,0.050000,0.220361,0.354381,0.409794,0.500000,0.349061,0.302327,0.325449
400,No log,No log,0.212629,0.362113,0.417526,0.534794,0.212629,0.120704,0.083505,0.053479,0.212629,0.362113,0.417526,0.534794,0.359134,0.305256,0.325327,0.211340,0.351804,0.409794,0.525773,0.211340,0.117268,0.081959,0.052577,0.211340,0.351804,0.409794,0.525773,0.353704,0.300737,0.321534


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

## Task 5: Evaluating our Retriever

Now that we have fine-tuned our retriever - let's see if it's worthwhile!

We'll start with some basic imports.

In [20]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

Now we'll define a function that will help us evaluate our retrieval process.

> NOTE: We're assuming 1 correct document in a "hit".

In [24]:
import tqdm

def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm.tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

All that's left to do is evaluate, we'll evaluate our model against:

1. OpenAI's closed source `text-embedding-3-small`
2. The base non-fine-tuned version of `Snowflake/snowflake-arctic-embed-m`.

Let's see how it stacks up!

### `text-embedding-3-small`

In [ ]:
te3_openai = OpenAIEmbeddings(model="text-embedding-3-small")
te3_results = evaluate_openai(test_dataset, te3_openai)

100%|██████████| 100/100 [00:19<00:00,  5.25it/s]


In [ ]:
te3_results_df = pd.DataFrame(te3_results)

In [ ]:
te3_hit_rate = te3_results_df["is_hit"].mean()
te3_hit_rate

1.0

### `Snowflake/snowflake-arctic-embed-m` (base)

In [25]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 663/663 [00:05<00:00, 121.80it/s]


In [26]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [27]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

0.6003016591251885

### `Snowflake/snowflake-arctic-embed-m` (fine-tuned)

In [31]:
finetune_embeddings = HuggingFaceEmbeddings(model_name="finetuned-v2-distiluse-base-multilingual-cased-v1")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)

100%|██████████| 663/663 [00:06<00:00, 97.23it/s] 


In [32]:
finetune_results_df = pd.DataFrame(finetune_results)

In [33]:
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

0.5444947209653092

In [36]:
bge_small_id = "BAAI/bge-small-en-v1.5"
bge_small = HuggingFaceEmbeddings(model_name = bge_small_id)

In [37]:
bge_small_m_results = evaluate_openai(test_dataset, bge_small)
bge_small_m_results_df = pd.DataFrame(bge_small_m_results)
bge_small_m_hit_rate = bge_small_m_results_df["is_hit"].mean()
bge_small_m_hit_rate

100%|██████████| 663/663 [00:09<00:00, 68.33it/s]


0.5052790346907994

In [38]:
qa_mp_net_m_results = evaluate_openai(test_dataset, HuggingFaceEmbeddings(model_name = 'sentence-transformers/multi-qa-mpnet-base-cos-v1'))
qa_mp_net_m_results_df = pd.DataFrame(qa_mp_net_m_results)
qa_mp_net_m_hit_rate = qa_mp_net_m_results_df["is_hit"].mean()
qa_mp_net_m_hit_rate

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 663/663 [00:10<00:00, 63.86it/s]


0.49924585218702866

In [41]:
mxbai_m_results = evaluate_openai(test_dataset, HuggingFaceEmbeddings(model_name ='mixedbread-ai/mxbai-embed-large-v1'))
mxbai_m_results_df = pd.DataFrame(mxbai_m_results)
mxbai_m_hit_rate = mxbai_m_results_df["is_hit"].mean()
mxbai_m_hit_rate

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

100%|██████████| 663/663 [00:17<00:00, 38.92it/s]


0.5143288084464555

In [43]:

MiniLM_results = evaluate_openai(test_dataset, HuggingFaceEmbeddings(model_name ='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'))
MiniLM_results_df = pd.DataFrame(MiniLM_results)
MiniLM_hit_rate = MiniLM_results_df["is_hit"].mean()
MiniLM_hit_rate

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 663/663 [00:08<00:00, 76.39it/s]


0.5158371040723982

In [44]:
BERITMBAU_results = evaluate_openai(test_dataset, HuggingFaceEmbeddings(model_name ='rufimelo/Legal-BERTimbau-sts-large'))
BERITMBAU_results_df = pd.DataFrame(BERITMBAU_results)
BERITMBAU_hit_rate = BERITMBAU_results_df["is_hit"].mean()
BERITMBAU_hit_rate

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.80k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Invalid model-index. Not loading eval results into CardData.


1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.
100%|██████████| 663/663 [00:16<00:00, 40.78it/s]


0.4298642533936652

In [45]:
TESTE_results = evaluate_openai(test_dataset, HuggingFaceEmbeddings(model_name ='winderfeld/cc-uffs-ppc-distiluse-base-multilingual-cased-v1-finetuned'))
TESTE_results_df = pd.DataFrame(TESTE_results)
TESTE_hit_rate = TESTE_results_df["is_hit"].mean()
TESTE_hit_rate

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/93.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

100%|██████████| 663/663 [00:05<00:00, 121.41it/s]


0.583710407239819

In [47]:
test1_results = evaluate_openai(test_dataset, HuggingFaceEmbeddings(model_name ='winderfeld/cc-uffs-ppc'))
test1_results_df = pd.DataFrame(test1_results)
test1_hit_rate = test1_results_df["is_hit"].mean()
test1_hit_rate

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/93.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

100%|██████████| 663/663 [00:09<00:00, 73.33it/s]


0.5052790346907994

In [49]:
test2_results = evaluate_openai(test_dataset, HuggingFaceEmbeddings(model_name ='paraphrase-multilingual-mpnet-base-v2'))
test2_results_df = pd.DataFrame(test2_results)
test2_hit_rate = test2_results_df["is_hit"].mean()
test2_hit_rate

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 663/663 [00:11<00:00, 60.19it/s]


0.5535444947209653

In [52]:
test2_results = evaluate_openai(test_dataset, HuggingFaceEmbeddings(model_name ='hkunlp/instructor-xl'))
test2_results_df = pd.DataFrame(test2_results)
test2_hit_rate = test2_results_df["is_hit"].mean()
test2_hit_rate

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

100%|██████████| 663/663 [00:33<00:00, 20.03it/s]


0.5520361990950227

In [53]:

test2_results = evaluate_openai(test_dataset, HuggingFaceEmbeddings(model_name ='intfloat/multilingual-e5-large-instruct'))
test2_results_df = pd.DataFrame(test2_results)
test2_hit_rate = test2_results_df["is_hit"].mean()
test2_hit_rate

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

100%|██████████| 663/663 [00:20<00:00, 32.51it/s]


0.526395173453997

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from huggingface_hub import HfApi, Repository
from google.colab import userdata
# Define repo name and directory
repo_name = "cc-uffs-ppc-ft-test"
local_model_dir = "finetuned-v2-distiluse-base-multilingual-cased-v1"

# Create or push to the repo
api = HfApi()
repo_url = api.create_repo(repo_name, token = userdata.get('HF_TOKEN'), private=False, exist_ok=True)

# Use HfApi to upload files directly
api.upload_folder(
    token = userdata.get('HF_TOKEN'),
    folder_path=local_model_dir,
    path_in_repo="",
    repo_id=f"winderfeld/{repo_name}",
    repo_type="model"
)




## Vibe Checking the RAG Pipeline

We're going to use our RAG pipeline to vibe check on some common phrases now that we've modified it!

### Creating New Chunks

In order to try and evaluate our system more fairly, let's create new chunks that we will use to create our Vector Store.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 50,
    length_function = len
)

training_documents = text_splitter.split_documents(training_documents_loaded.load())

### Base Chain

We'll start by constructing our base chain, which will use the untrained retrieval model.

#### R - Retrieval

In [ ]:
from langchain_community.vectorstores import FAISS

base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

#### A - Augmented

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

#### G - Generation

In [ ]:
rag_llm =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

#### RAG - LCEL RAG Pipeline

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [ ]:
base_rag_chain.invoke({"question" : "What is Ayahuasca?"})["response"]

'I do not know.'

In [ ]:
base_rag_chain.invoke({"question" : "What is the difference between Neurosurgery vs. Neuralink surgery?"})["response"]

'The difference between neurosurgery and Neuralink surgery lies primarily in the approach and risk involved. Traditional neurosurgery often involves more invasive procedures, such as opening deeper parts of the brain or manipulating blood vessels, which carry significant risks. In contrast, Neuralink surgery involves cortical micro-insertions that are performed on the surface of the brain, which significantly reduces the risk compared to more invasive surgeries like those for tumors or aneurysms. Additionally, Neuralink aims to utilize advanced technology and robotics to enhance precision in electrode placement, potentially changing the landscape of neurosurgical practices.'

In [ ]:
base_rag_chain.invoke({"question" : "What is Neural Dust?"})["response"]

'I do not know.'

In [ ]:
base_rag_chain.invoke({"question" : "What is a Neural Decoder?"})["response"]

'A Neural Decoder is a system or algorithm that interprets neural signals, such as sequences of spikes from neurons, to extract meaningful information or predictions. It involves machine learning techniques to create a mapping between the neural data and specific outputs or labels, addressing challenges in architecture and hyperparameters to optimize performance.'

In [ ]:
base_rag_chain.invoke({"question" : "Who got Neuralink surgery?"})["response"]

'The first human being to receive Neuralink surgery is referred to as Noland.'

### Fine-tuned Embedding Model

Now let's rebuild our RAG chain with the Fine-tuned model - the only component we need to change is our `FAISS` vectorstore!

In [ ]:
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

In [ ]:
finetune_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [ ]:
finetune_rag_chain.invoke({"question" : "What is Ayahuasca?"})["response"]

'Ayahuasca is a psychoactive brew made from the Banisteriopsis caapi vine and other ingredients, often used in traditional Amazonian shamanic practices for spiritual and healing purposes. It contains DMT (dimethyltryptamine), a powerful hallucinogenic compound, which can induce altered states of consciousness and profound experiences.'

In [ ]:
finetune_rag_chain.invoke({"question" : "What is the difference between Neurosurgery vs. Neuralink surgery?"})["response"]

'Neurosurgery typically involves more complex procedures that may include opening deeper parts of the brain or manipulating blood vessels, which carries significant risks. In contrast, Neuralink surgery focuses on making cortical micro-insertions on the surface of the brain, which is considered to carry significantly less risk compared to traditional neurosurgeries like tumor or aneurysm surgeries. Additionally, Neuralink has developed a rigorous practice regimen, including hundreds of animal surgeries and the use of lifelike models to simulate the procedure before performing it on humans.'

In [ ]:
finetune_rag_chain.invoke({"question" : "What is Neural Dust?"})["response"]

'Neural dust refers to a technology that involves tiny, wireless sensors that can be implanted in the brain to monitor neural activity. These sensors are designed to communicate with external devices, allowing for real-time data collection and potentially enabling advanced brain-computer interfaces.'

In [ ]:
finetune_rag_chain.invoke({"question" : "What is a Neural Decoder?"})["response"]

'A Neural Decoder is a system or model that interprets and translates neural signals, such as sequences of spikes from neurons, into meaningful outputs or predictions. It involves building a dataset, optimizing labels for the model, and determining the appropriate architecture and hyperparameters to effectively map the input data to the desired output. The process combines elements of machine learning, science, and art to create a reliable and efficient decoding mechanism.'

In [ ]:
finetune_rag_chain.invoke({"question" : "Who got Neuralink surgery?"})["response"]

'Noland Arbaugh got Neuralink surgery.'

## RAGAS Evaluation

It's great to have some idea of how our system is doing based on vibe-checks, but let's use RAGAS to provide more insight info. on how things are improving!

In [ ]:
!pip install -qU ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.2 MB/s eta 0:00:00


### RAGAS Synthetic Testset Generation

First things first, we need to generate some data to test our model on.

Let's use our test data that we created before as a base!

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

In [ ]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

In [ ]:
testset = generator.generate_with_langchain_docs(test_split_documents, test_size=20, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

embedding nodes:   0%|          | 0/100 [00:00<?, ?it/s]

Generating:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
testset.to_pandas().head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How does the variability of tetraplegia situat...,"[DJ Seo (03:27:14) Yeah, I mean the primary go...",The variability of tetraplegia situations impa...,simple,"[{'source': 'elon_lex_transcript.html.html', '...",True
1,What is the potential future scenario of havin...,"[Lex Fridman (03:37:00) And then, you’d be con...",The potential future scenario is that there co...,simple,"[{'source': 'elon_lex_transcript.html.html', '...",True
2,What was Matthew MacDougall's initial focus wh...,"[Matthew MacDougall (03:45:07) Basically, the ...",Matthew MacDougall's initial focus when lookin...,simple,"[{'source': 'elon_lex_transcript.html.html', '...",True
3,What led the speaker to decide to study the br...,"[Lex Fridman (03:48:25) So, from there, the ea...",The speaker decided to study the brain because...,simple,"[{'source': 'elon_lex_transcript.html.html', '...",True
4,How does being nervous while talking affect pe...,[Lex Fridman (03:26:05) … a lot of novel insig...,Being nervous while talking can affect perform...,simple,"[{'source': 'elon_lex_transcript.html.html', '...",True


### Generating Answer Datasets

For each of our pipelines, let's generate answers to these questions!

Once we have our: Questions, Answers, Contexts, Ground Truths we can move on to evaluating our datasets!

In [ ]:
from datasets import Dataset

def generate_answers(chain, testset):
  answers = []
  contexts = []
  questions = testset.to_pandas()["question"].values.tolist()
  ground_truths = testset.to_pandas()["ground_truth"].values.tolist()

  for question in tqdm.tqdm(questions):
    answer = chain.invoke({"question" : question})
    answers.append(answer["response"])
    contexts.append([context.page_content for context in answer["context"]])

  return Dataset.from_dict({
      "question" : questions,
      "answer" : answers,
      "contexts" : contexts,
      "ground_truth" : ground_truths
  })

In [ ]:
base_dataset = generate_answers(base_rag_chain, testset)

100%|██████████| 20/20 [00:25<00:00,  1.28s/it]


In [ ]:
finetune_dataset = generate_answers(finetune_rag_chain, testset)

100%|██████████| 20/20 [00:29<00:00,  1.48s/it]


### Evaluating Using the Test Set

Now that we have a test set - it's time to evaluate our pipelines with it!

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [ ]:
from ragas import evaluate

result = evaluate(
    base_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
result

{'context_precision': 0.5128, 'faithfulness': 0.4737, 'answer_relevancy': 0.4764, 'context_recall': 0.4583}

In [ ]:
result.to_pandas().head()

,question,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,How does the variability of tetraplegia situat...,I do not know.,"[Future Neuralink capabilities, that you can e...",The variability of tetraplegia situations impa...,1.0,0.0,0.000000,0.0
1,What is the potential future scenario of havin...,I do not know.,"[Future Neuralink capabilities, experience, I’...",The potential future scenario is that there co...,0.0,0.0,0.000000,0.0
2,What was Matthew MacDougall's initial focus wh...,Matthew MacDougall's initial focus when lookin...,[Matthew MacDougall (03:48:34) Yeah. It was so...,Matthew MacDougall's initial focus when lookin...,1.0,1.0,0.999999,0.0
3,What led the speaker to decide to study the br...,The speaker was influenced to study the brain ...,"[into delving deep into the human brain, so no...",The speaker decided to study the brain because...,1.0,1.0,0.977114,0.0
4,How does being nervous while talking affect pe...,I do not know.,"[point, muscle memory kicks in and you sort of...",Being nervous while talking can affect perform...,1.0,0.0,0.000000,0.0


In [ ]:
result = evaluate(
    finetune_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
result

{'context_precision': 0.8086, 'faithfulness': 0.8799, 'answer_relevancy': 0.8680, 'context_recall': 0.9667}

In [ ]:
result.to_pandas().head()

,question,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,How does the variability of tetraplegia situat...,The variability of tetraplegia situations impa...,"[DJ Seo (03:27:14) Yeah, I mean the primary go...",The variability of tetraplegia situations impa...,0.966667,1.000000,1.000000,1.0
1,What is the potential future scenario of havin...,The potential future scenario suggests that th...,[Lex Fridman (00:31:41) And that output rate w...,The potential future scenario is that there co...,1.000000,0.800000,0.000000,1.0
2,What was Matthew MacDougall's initial focus wh...,Matthew MacDougall's initial focus when lookin...,[Matthew MacDougall (03:48:34) Yeah. It was so...,Matthew MacDougall's initial focus when lookin...,1.000000,1.000000,1.000000,1.0
3,What led the speaker to decide to study the br...,The speaker was influenced by personal experie...,"[(01:29:17) But also at the same time, I think...",The speaker decided to study the brain because...,1.000000,1.000000,0.977114,1.0
4,How does being nervous while talking affect pe...,Being nervous while talking can negatively imp...,"[point, muscle memory kicks in and you sort of...",Being nervous while talking can affect perform...,0.700000,0.428571,0.997113,1.0


## Conclusion

As you can see - with only a few hundred data points, we're able to increase our embedding model and increase the effectiveness of RAG!